In [14]:
# import scipy
# import statsmodels.api as sm
# !python -m pip install -Iv scipy==1.2 --upgrade
scipy.__version__


'1.3.0'

In [15]:
from __future__ import print_function
import numpy as np
import statsmodels.api as sm
import pandas as pd
from statsmodels.tsa.arima_process import arma_generate_sample

ImportError: cannot import name 'factorial'

In [3]:
import numpy as np
np.random.seed(12345)
arparams = np.array([.75, -.25])
maparams = np.array([.65, .35])
ar = np.r_[1, -arparams] # add zero-lag and negate
ma = np.r_[1, maparams] # add zero-lag
y = sm.tsa.arma_generate_sample(ar, ma, 250)
model = sm.tsa.ARMA(y, (2, 2)).fit(trend='nc', disp=0)
model.params

NameError: name 'sm' is not defined